In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6"  color="red">ch15. 데이터베이스 연동 </font></b>
# 1절 SQLite데이터베이스 연결
- SQL라이트는 별도의 DBMS없이 SQL문을 이용해서 DB액세스할 수 있도록 만든 간단한 디스크 기반 DB(실제 프로젝트 이외 시제품이나 내부 실험용 사용)
- C라이브러리
- SQLite는 프로토타입 생성시 사용
- 프로젝트 : 분석 - > 설계 - > 구현 -> 테스트 -> 고객에게 배포 -> 유지보수
          프로토타입(SQLite)         완제품(Oracle,MySQL,Maria,...)
- [DB browser for SQLite] (https://sqlitebrowser.org/dl/)

## 1.1 SQLite browser 설치 및 sqlite 패키지 load

In [14]:
import sqlite3
sqlite3.sqlite_version

'3.40.1'

In [15]:
import pandas as pd
pd.__version__

'1.5.3'

## 1.2 데이터베이스 연결
```
(1) 데이터베이스 연결 생성
(2) SQL 전송 객체 생성(cursor)
(3) SQL 전송 & 결과 받기 (cursor.execute()이용)
(4) cursor 해제 & 데이터베이스 연결 객체 해제
```

## (1) DB 연결 : sqlite로 연결시 DB파일이 없으면 빈 DB파일을 생성하고 연결,  있으면 연결

In [16]:
conn = sqlite3.connect('data/ch15_example.db')
conn

## (2) SQL 전송 객체 생성(cursor 객체)

In [17]:
cursor = conn.cursor()
cursor

In [18]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

OperationalError: table MEMBER already exists

In [19]:
cursor.execute('DROP TABLE MEMBER')

OperationalError: database is locked

In [20]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

OperationalError: table MEMBER already exists

In [10]:
cursor.execute('INSERT INTO MEMBER VALUES (\"홍길동\",20,\"h@h.com\")')
print('수행결과 행수 :',cursor.rowcount)

OperationalError: database is locked

In [ ]:
sql = "INSERT INTO MEMBER VALUES ('김길동',30,'K@K.COM')"
cursor.execute(sql)
print('수행결과 행수 :', cursor.rowcount)
cursor.execute("INSERT INTO MEMBER VALUES ('이길동',25,'l@l.COM')")
print('수행결과 행수 :', cursor.rowcount)

In [ ]:
conn.commit()  # (반) conn.rollback()

## (3) select 전송 결과 : CURSOR가 가리킴

In [11]:
cursor.execute("SELECT * FROM MEMBER")

In [28]:
# insert, update, delete문 실행결과 : cursor.rowcount
# select문 실행 결과 : 
    ## fetchall(): 결과를 모두 받을 때 (튜플 list)
    ## fetchone(): 결과를 한행씩 받을 때(튜플)
    ## fetchmany(n) : 결과를 n행 받을 때 (튜플 list)
print(cursor.fetchall())

[]


In [23]:
print(cursor.fetchall()) # 한번 소요된 cursor 객체는 다시 fetch될 수 없음

[]


In [29]:
# 한꺼번에 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members

[('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('이길동', 25, 'l@l.COM'),
 ('김길동', 30, 'K@K.COM')]

In [25]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('홍길동', 20, 'h@h.com')
('홍길동', 20, 'h@h.com')
('이길동', 25, 'l@l.COM')
('김길동', 30, 'K@K.COM')


In [30]:
# 하나씩 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = []
while True:
    member = cursor.fetchone()
    if member is None:
        print('데이터 끝')
        break
    members.append(member)


데이터 끝


In [27]:
members

[('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('이길동', 25, 'l@l.COM'),
 ('김길동', 30, 'K@K.COM')]

In [31]:
# 최상위 n행 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'h@h.com')
('홍길동', 20, 'h@h.com')


In [32]:
cursor.description

(('NAME', None, None, None, None, None, None),
 ('AGE', None, None, None, None, None, None),
 ('EMAIL', None, None, None, None, None, None))

In [37]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self,name,age,email):
        self.name = name
        self.age = age
        self.email =email
    def __str__(self):
        return "{}\t{}\t{}".format(self.name,self.age,self.email)
def to_member(*row):  # 튜플매개변수를 받아 Member형 객체를 return
    return Member(row[0],row[1],row[2])

In [39]:
member = to_member('홍길동',20,'h@h.com')
print(member)

홍길동	20	h@h.com


In [42]:
cursor.execute('SELECT * FROM MEMBER')
member_list = [] # sql문 수행한 결과를 담을 객체 list
members = cursor.fetchall() #튜플 list
print(members)
for member in members:
    member_list.append(to_member(*member))

[('홍길동', 20, 'h@h.com'), ('홍길동', 20, 'h@h.com'), ('홍길동', 20, 'h@h.com'), ('김길동', 30, 'K@K.COM'), ('이길동', 25, 'l@l.COM')]


In [55]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('홍길동', 20, 'h@h.com')
('홍길동', 20, 'h@h.com')
('김길동', 30, 'K@K.COM')
('이길동', 25, 'l@l.COM')


## (4) 연결해제

In [56]:
cursor.close()
conn.close()

## 1.3 SQL 구문에 <font color="red"> 파라미터 사용하기</font>
- qmark(DB에 따라 불가한 경우 있음)
- named(추천)

In [26]:
import sqlite3
conn = sqlite3.connect('data/ch15_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동','김길동')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('김길동', 30, 'K@K.COM'),
 ('홍길동', 22, 'h@h.com')]

In [3]:
#파라미터 사용하기 : qmark 방법
name1 = input('검색할 이름1?')
name2 = input('검색할 이름2?')
#cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('"+name1+"','"+name2+"')")
#cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('{name1}','{name2}')")
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)",(name1,name2))
cursor.fetchall()

검색할 이름1?홍길동
검색할 이름2?김길동


[('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('김길동', 30, 'K@K.COM')]

In [4]:
## 파라미터 사용하기 : * named 방법
name1 = input('검색할 이름1?')
name2 = input('검색할 이름2?')
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1,:name2)",{'name1':name1,'name2':name2})
cursor.fetchall()

검색할 이름1?홍길동
검색할 이름2?김길동


[('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('홍길동', 20, 'h@h.com'),
 ('김길동', 30, 'K@K.COM')]

In [5]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self,name,age,email):
        self.name = name
        self.age = age
        self.email =email
    def __str__(self):
        return "{}\t{}\t{}".format(self.name,self.age,self.email)
def to_member(*row):  # 튜플매개변수를 받아 Member형 객체를 return
    return Member(row[0],row[1],row[2])

##  *Member 테이블에 입력(사용자로부터 이름,나이,메일을 받아 insert)

In [11]:
# Member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 받아 insert)
try :
    name = input('입력할 이름 :')
    age  = int(input('입력할 나이 :'))
except:
    print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')
inputdata = {'name':name, 'age':age, 'email':email} # named방식에서 사용
inputdata2 = (name, age, email) # qmark방식에서 사용
sql = "INSERT INTO MEMBER VALUES (:name, :age, :email)"
cursor.execute(sql, inputdata)
conn.commit()
if cursor.rowcount:
    print('저장완료')

입력할 이름 :ㅇㄴ
입력할 나이 :ㄴㅇ
유효하지 않은 나이를 입력한 경우, 18세로 초기화
입력할 메일은 :
저장완료


In [14]:
# Member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 받아 insert)
try :
    name = input('입력할 이름 :')
    age  = int(input('입력할 나이 :'))
except:
    print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')
newMember = Member(name, age, email)
# print(newMember)
# print(newMember.__dict__)
sql = "INSERT INTO MEMBER VALUES (:name,:age,:email)"
cursor.execute(sql,newMember.__dict__)
conn.commit()
if cursor.rowcount==1:
    print('입력성공')

입력할 이름 :유관순
입력할 나이 :a
유효하지 않은 나이를 입력한 경우, 18세로 초기화
입력할 메일은 :y@y.com
입력성공


In [17]:
while True:   
    try :
        name = input('입력할 이름(종료는 0) :')
        if name=='0':
            break;
        age  = int(input('입력할 나이 :'))
    except:
        print('유효하지 않은 나이를 입력한 경우, 18세로 초기화')
        age = 18
    email = input('입력할 메일은 :')
    newMember = Member(name, age, email)
    # print(newMember)
    # print(newMember.__dict__)
    sql = "INSERT INTO MEMBER VALUES (:name,:age,:email)"
    cursor.execute(sql,newMember.__dict__)
    if cursor.rowcount==1:
        print('입력성공')
conn.commit()

입력할 이름(종료는 0) :sdf
입력할 나이 :sDF
유효하지 않은 나이를 입력한 경우, 18세로 초기화
입력할 메일은 :SDF
입력성공
입력할 이름(종료는 0) :SF
입력할 나이 :44
입력할 메일은 :ds
입력성공
입력할 이름(종료는 0) :0


In [27]:
cursor.close()
conn.close()

# 2절 오라클 데이터베이스 연결
- pip install cx_oracle (oracle 11g까지)
- pip install oracledb (oracle 12부터)

In [28]:
import cx_Oracle
cx_Oracle.__version__

'8.3.0'

# *conn 얻어오는 방법1

In [30]:
conn = cx_Oracle.connect("scott","tiger","localhost:1521/xe")
cursor = conn.cursor()
sql = 'SELECT EMPNO "NO",ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO FROM EMP'
cursor.execute(sql)
emp = cursor.fetchall()
print(emp)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7900, 

In [27]:
# conn 얻어오는 방법2
import oracledb
oracledb.init_oracle_client()
# conn = oracledb.connect("scott/tiger@localhost:1521/xe")
conn = oracledb.connect(
    user="scott",
    password="tiger",
    host="localhost",
    port=1521,
    sid="xe"
)
cursor = conn.cursor()
sql = 'SELECT EMPNO "NO", ENAME, JOB, MGR, HIREDATE, SAL, COMM, DEPTNO FROM EMP'
cursor.execute(sql)
emp = cursor.fetchall()
print(emp)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7900, 

In [28]:
import pandas as pd
emp_df = pd.DataFrame(emp)
emp_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [29]:
# select문을 수행한 필드 정보(NO,ENAME,JOB,~)
cursor.description


[('NO', <DbType DB_TYPE_NUMBER>, 5, None, 4, 0, False),
 ('ENAME', <DbType DB_TYPE_VARCHAR>, 10, 10, None, None, True),
 ('JOB', <DbType DB_TYPE_VARCHAR>, 9, 9, None, None, True),
 ('MGR', <DbType DB_TYPE_NUMBER>, 5, None, 4, 0, True),
 ('HIREDATE', <DbType DB_TYPE_DATE>, 23, None, None, None, True),
 ('SAL', <DbType DB_TYPE_NUMBER>, 11, None, 7, 2, True),
 ('COMM', <DbType DB_TYPE_NUMBER>, 11, None, 7, 2, True),
 ('DEPTNO', <DbType DB_TYPE_NUMBER>, 3, None, 2, 0, True)]

In [30]:
[description[0] for description in cursor.description]

['NO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [31]:
emp_df.columns = [description[0] for description in cursor.description]
emp_df.head()

,NO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


#  *검색할 이름을 사용자에게 받아 해당 내용을 출력


In [33]:
# 검색할 이름을 사용자에게 받아 해당 내용을 출력
ename = input('검색할 이름은?').upper()
# print(ename)
sql = "SELECT * FROM EMP WHERE ENAME = :ename"
cursor.execute(sql, {'ename':ename})
emp = cursor.fetchall()
if emp:
    print(emp[0])
else:
    print('입력하신 이름의 데이터는 없습니다')

검색할 이름은?scott
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)


In [34]:
fieldnames = [descrip[0] for descrip in cursor.description]
fieldnames

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [36]:
for fieldname, data in zip(fieldnames, emp[0]):
    print("{}:{}".format(fieldname, data if data is not None else ' -'))

EMPNO:7788
ENAME:SCOTT
JOB:ANALYST
MGR:7566
HIREDATE:1982-12-09 00:00:00
SAL:3000.0
COMM: -
DEPTNO:20


In [40]:
cursor.close()
conn.close()

InterfaceError: DPY-1001: not connected to database

# 3절 MySQL 데이터베이스 연결
- pip install mysql-connector-python (공식 connector)
- pip install PyMySQL(경량)

In [1]:
%pip install PyMySQL

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pymysql
conn = pymysql.connect(
    host = "localhost",
    user="root",
    password="mysql",
    db="devdb",
    charset="utf8mb4"
)
cursor = conn.cursor()
sql = "SELECT * FROM PERSONAL"
cursor.execute(sql)
personal = cursor.fetchall()
fieldnames = [descript[0] for descript in cursor.description]
pd.DataFrame(personal, columns=fieldnames)

OperationalError: (1049, "Unknown database 'devdb'")

In [6]:
cursor.close()
conn.close()

NameError: name 'cursor' is not defined

# 4절 연습문제
## 실습형
-회원가입|전체조회|이름찾기|메일삭제|csv보내기|종료

## 0.처음실행

In [8]:
# ORACLE SQL에서 회원가입|전체조회|이름찾기|메일삭제|csv보내기|종료 해당기능 QUERY 작성 

In [13]:
def load_cursor():
    global conn
    import oracledb
    oracledb.init_oracle_client()
    conn = oracledb.connect("scott/tiger@localhost:1521/xe")
load_cursor()

### 1. 입력

In [19]:
def fn1_insert_member_info():
    '입력받은 데이터를 member테이블에 입력'
    cursor = conn.cursor()
    name = input('이름 :')
    tel  = input('전화 :')
    email = input('이메일 :')
    try:
        age = int(input('나이 :'))
        if age>200:
            age = 200
        elif age<0:
            age = 0
    except:
        print('유효하지 않은 나이 입력시 나이는 0으로 초기화')
        age = 0
    try:
        grade = int(input('등급 :'))
        if grade<1:
            grade = 1
        elif grade>5:
            grade = 5
    except:
        print('유효하지 않은 등급 입력시 등급은 1로 초기화')
        grade = 1
    etc = input('기타정보 :')
    sql = "INSERT INTO MEMBER VALUES (:name, :tel, :email, :age, :grade, :etc)"
    inputdata = {'name':name, 'tel':tel, 'email':email, 'age':age, 'grade':grade, 'etc':etc}
    cursor.execute(sql, inputdata)
    conn.commit()
    if cursor.rowcount==1:
        print(name, '님 가입 완료')
    cursor.close()
fn1_insert_member_info()

이름 :홍길길
전화 :010-1111-1111
이메일 :a@aa.com
나이 :2
등급 :22
기타정보 :etc
홍길길 님 가입 완료


### 2.전체조회

In [26]:
import pandas as pd # pandas를 임포트합니다.

def fn2_print_members():
    'member테이블의 내용을 데이터프레임으로 display'
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER ORDER BY NAME" # 모든 멤버를 이름 순으로 조회합니다.
    cursor.execute(sql)

    # 컬럼 이름을 가져와서 데이터프레임 헤더로 사용합니다.
    columns = [desc[0] for desc in cursor.description]
    
    # 모든 결과 데이터를 가져옵니다.
    members_data = cursor.fetchall()
    
    cursor.close()

    if members_data:
        df = pd.DataFrame(members_data, columns=columns)
        print("--- 전체 회원 목록 ---")
        print(df) # 데이터프레임을 출력합니다.
    else:
        print("조회할 회원 정보가 없습니다.")
    print("--------------------")


fn2_print_members()

--- 전체 회원 목록 ---
  NAME            TEL     EMAIL  AGE  GRADE  ETC
0  홍길길  010-1111-1111  a@aa.com    2      5  etc
--------------------


### 3. 이름조회

In [23]:
import pandas as pd # pandas를 다시 임포트합니다 (이 파일 상단에 한 번만 임포트해도 괜찮아요).

def fn3_search_member():
    '찾고자하는 이름을 input으로 받아 해당 내용을 데이터프레임으로 display'
    cursor = conn.cursor()
    search_name = input('찾으시는 회원의 이름을 입력하세요: ') # 사용자로부터 이름을 입력받습니다.

    # SQL 쿼리를 사용하여 이름을 LIKE 연산자로 검색합니다.
    # 대소문자 구분 없이 검색하려면 LOWER() 함수를 사용하거나,
    # 정확히 일치하는 이름을 찾으려면 LIKE 대신 = 를 사용합니다.
    sql = "SELECT * FROM MEMBER WHERE NAME LIKE :search_name ORDER BY NAME"
    
    # 바인드 변수에 '%'를 포함하여 부분 일치 검색을 가능하게 합니다.
    cursor.execute(sql, {'search_name': '%' + search_name + '%'}) 

    columns = [desc[0] for desc in cursor.description]
    members_data = cursor.fetchall()
    
    cursor.close()

    if members_data:
        df = pd.DataFrame(members_data, columns=columns)
        print(f"--- '{search_name}' 검색 결과 ---")
        print(df)
    else:
        print(f"'{search_name}' 이름의 회원 정보가 없습니다.")
    print("--------------------")


fn3_search_member()

찾으시는 회원의 이름을 입력하세요: 홍길동
--- '홍길동' 검색 결과 ---
  NAME            TEL    EMAIL  AGE  GRADE  ETC
0  홍길동  010-9999-9999  H@H.COM   33      2  까칠해
--------------------


### 4. 메일 삭제

In [25]:
def fn4_delete_member():
    '삭제하고자 하는 메일을 input으로 받아 해당행을 삭제'
    cursor = conn.cursor()
    email_to_delete = input('삭제할 회원의 이메일을 입력하세요: ') # 사용자로부터 이메일을 입력받습니다.

    sql = "DELETE FROM MEMBER WHERE EMAIL = :email_to_delete"
    
    try:
        cursor.execute(sql, {'email_to_delete': email_to_delete})
        conn.commit() # 변경사항을 데이터베이스에 반영합니다.
        
        if cursor.rowcount > 0: # 삭제된 행이 있는지 확인합니다.
            print(f"이메일 '{email_to_delete}'을(를) 가진 회원이 삭제되었습니다.")
        else:
            print(f"이메일 '{email_to_delete}'을(를) 가진 회원이 존재하지 않습니다.")
    except Exception as e:
        conn.rollback() # 오류 발생 시 변경사항을 되돌립니다.
        print(f"회원 삭제 중 오류가 발생했습니다: {e}")
    finally:
        cursor.close()


fn4_delete_member()

삭제할 회원의 이메일을 입력하세요: H@H.COM
이메일 'H@H.COM'을(를) 가진 회원이 삭제되었습니다.


### 5. csv내보내기

In [28]:

import pandas as pd
import datetime # 파일명에 시간 정보를 포함하기 위해 datetime 모듈을 임포트합니다.

def fn5_save_csv():
    'member 테이블 내용을 csv로 내보내기'
    cursor = conn.cursor()
    sql = "SELECT NAME, TEL, EMAIL, AGE, GRADE, ETC FROM MEMBER ORDER BY NAME" # 모든 회원 정보를 조회합니다.
    cursor.execute(sql)

    columns = [desc[0] for desc in cursor.description] # 컬럼 이름을 가져옵니다.
    members_data = cursor.fetchall() # 모든 데이터를 가져옵니다.
    cursor.close()

    if members_data:
        df = pd.DataFrame(members_data, columns=columns)
        
        # 현재 시간을 파일명에 포함하여 저장합니다.
        # 예: members_20251119_174403.csv (현재 2025. 11. 19. 오후 5:44:03 기준)
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"members_{now}.csv"
        
        # DataFrame을 CSV 파일로 저장합니다. index=False는 인덱스 번호를 CSV에 저장하지 않도록 합니다.
        df.to_csv(filename, index=False, encoding='utf-8') 
        print(f"회원 정보가 '{filename}' 파일로 성공적으로 저장되었습니다.")
    else:
        print("CSV로 내보낼 회원 정보가 없습니다.")
    print("--------------------")


fn5_save_csv()

회원 정보가 'members_20251119_174443.csv' 파일로 성공적으로 저장되었습니다.
--------------------


In [ ]:
load_cursor()
while True:
    menu = input("1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료")
    if menu=='1':
        fn1_insert_member_info()
    elif menu=='2':
        fn2_print_members()
    elif menu=='3':
        fn3_search_member()
    elif menu=='4':
        fn4_delete_member()
    elif menu=='5':
        fn5_save_csv()
    elif menu=='0':
        conn.close()
        break
    else:
        print('유효한 메뉴 번호를 입력해 주세요')

1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료1
이름 :홍길동
전화 :010-2222-2222
이메일 :H@H.COM
나이 :23
등급 :2
기타정보 :ETC
홍길동 님 가입 완료
1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료2
--- 전체 회원 목록 ---
  NAME            TEL     EMAIL  AGE  GRADE  ETC
0  홍길길  010-1111-1111  a@aa.com    2      5  etc
1  홍길동  010-2222-2222   H@H.COM   23      2  ETC
--------------------
1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료3
찾으시는 회원의 이름을 입력하세요: 홍길동
--- '홍길동' 검색 결과 ---
  NAME            TEL    EMAIL  AGE  GRADE  ETC
0  홍길동  010-2222-2222  H@H.COM   23      2  ETC
--------------------
1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료4
삭제할 회원의 이메일을 입력하세요: 홍길동
이메일 '홍길동'을(를) 가진 회원이 존재하지 않습니다.
1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료H@H.COM
유효한 메뉴 번호를 입력해 주세요
1.회원입력 | 2.전체조회 | 3.이름찾기 | 4.메일삭제 | 5.CSV내보내기 | 0.종료2
--- 전체 회원 목록 ---
  NAME            TEL     EMAIL  AGE  GRADE  ETC
0  홍길길  010-1111-1111  a@aa.com    2      5  etc
1  홍길동  010-2222-2222   H@H.COM   23      2  ETC
-----------